In [2]:
import pandas as pd
import seaborn as sns

%load_ext google.cloud.bigquery

# Channel KPI Version

In [4]:
params = {
    'report_start_date': '2022-10-01',
    'report_end_date': '2022-10-31'
}

In [232]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Unsubs_v4_Oct22_test` AS (
    SELECT DISTINCT adobe_tracking_id
    FROM  `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Unsubs`
    WHERE first_unsub_date <= @report_end_date
)

Query is running:   0%|          |

""


In [233]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Delivered_v4_Oct22_test` AS ( -- everyone who have received emails in the month
    SELECT  distinct adobe_tracking_id
    FROM
    (
        SELECT  DISTINCT adobe_tracking_id
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
        WHERE event_name = 'Email Deliveries'
        AND event_date BETWEEN @report_start_date AND @report_end_date
        AND LOWER(campaign_name) NOT LIKE 'transactional%' -- Exclude transactional emails 
    ) b
    -- exclude users put in holdout group
    LEFT JOIN
    (
        SELECT  DISTINCT TrackingId AS aid
        FROM `nbcu-ds-prod-001.PeacockDataMartMarketingGold.HOLDOUT_GROUP`
        WHERE cohort = format_timestamp('%B%Y', DATETIME_TRUNC(@report_start_date, QUARTER))
        AND Hold_Out_Type_Current = 'Owned Email Holdout'
        AND DATE(TIMESTAMP(RegistrationDate), 'America/New_York') <= @report_end_date 
    ) g
    ON g.aid = b.adobe_tracking_id
    WHERE g.aid is null
)

Query is running:   0%|          |

""


In [234]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Holdout_v4_Oct22_test` AS ( 
    SELECT  DISTINCT holdout.adobe_tracking_id
    FROM
    (
        SELECT  DISTINCT TrackingId AS adobe_tracking_id
        FROM `nbcu-ds-prod-001.PeacockDataMartMarketingGold.HOLDOUT_GROUP`
        WHERE cohort = format_timestamp('%B%Y', DATETIME_TRUNC(@report_start_date, QUARTER)) -- get cohort name as month of quarter start + year
        AND Hold_Out_Type_Current = 'Owned Email Holdout'
        AND DATE(TIMESTAMP(RegistrationDate), 'America/New_York') <= @report_end_date
    ) holdout
    -- Exclude those who are assigned to Email Holdout but actually received emails in holdout period
    LEFT JOIN (
        SELECT DISTINCT adobe_tracking_id 
        FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
        WHERE event_name = 'Email Deliveries' 
        AND event_date BETWEEN DATETIME_TRUNC(@report_start_date, QUARTER) and @report_end_date
        AND LOWER(campaign_name) NOT LIKE 'transactional%' -- Exclude transactional emails
    ) received
    ON holdout.adobe_tracking_id = received.adobe_tracking_id
    WHERE received.adobe_tracking_id IS NULL
)

Query is running:   0%|          |

""


In [235]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Channel_Qualifier_v4_Oct22_test` AS 

-- Engagement: Deliveries 4 months before start of the holdout period, defined as start of quarter
SELECT DISTINCT adobe_tracking_id AS aid 
FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
WHERE event_name = 'Email Deliveries'
AND event_date BETWEEN DATE_SUB(DATETIME_TRUNC(@report_start_date, QUARTER), INTERVAL 4 MONTH) AND @report_end_date
AND lower(campaign_name) NOT LIKE 'transactional%' -- Exclude transactional emails

UNION ALL

-- New users joining after 4 months before start of the cohort period
SELECT DISTINCT adobe_tracking_id AS aid
FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
WHERE registration_date BETWEEN DATE_SUB(DATETIME_TRUNC(@report_start_date, QUARTER), INTERVAL 4 MONTH) AND @report_end_date

Query is running:   0%|          |

""


### Output Base Tables

In [236]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Measurement_Audience_v4_Oct22_test` AS

SELECT  distinct delivered_and_holdout.adobe_tracking_id AS aid
       ,cohort
       ,user.account_type
       ,user.account_tenure
FROM (
    SELECT *, 'Email_Targeted' as cohort from `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Delivered_v4_Oct22_test`
    UNION ALL
    SELECT *, 'Holdout' as cohort from `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Holdout_v4_Oct22_test`
) delivered_and_holdout

-- Include only those who received email in the current reporting period or are in holdout
INNER JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Channel_Qualifier_v4_Oct22_test` qualified
ON delivered_and_holdout.adobe_tracking_id = qualified.aid

-- for after 2021/july, email channel only, take out all abandon MAAs
INNER JOIN
    (
        SELECT  DISTINCT adobe_tracking_id
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES`
        WHERE report_date = @report_end_date
        AND date_of_last_view IS NOT NULL
    ) abandon_maa
ON delivered_and_holdout.adobe_tracking_id = abandon_maa.adobe_tracking_id

--add attribute: account_type at the end of the reporting period
INNER JOIN
    (
        SELECT  DISTINCT adobe_tracking_id
            ,account_type
            ,account_tenure
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
        WHERE report_date = @report_end_date 
    ) user
ON delivered_and_holdout.adobe_tracking_id = user.adobe_tracking_id

-- exclude unsubscribed
LEFT JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Unsubs_v4_Oct22_test` email_unsubs
ON delivered_and_holdout.adobe_tracking_id = email_unsubs.adobe_tracking_id
WHERE email_unsubs.adobe_tracking_id IS NULL

Query is running:   0%|          |

""


### Audience total size

In [237]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
SELECT cohort, count(distinct aid) from `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Measurement_Audience_v4_Oct22_test` group by 1

Query is running:   0%|          |

Downloading:   0%|          |

,cohort,f0_
0,Email_Targeted,40293952
1,Holdout,2018322


In [238]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
WITH 
base as (
    SELECT aid, cohort, paying_account_flag, billing_cycle
    from `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Measurement_Audience_v4_Oct22_test` a
    --add attribute: account_type at the end of the reporting period
    INNER JOIN
        (
            SELECT  DISTINCT adobe_tracking_id
                ,paying_account_flag
                ,billing_cycle
            FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
            WHERE report_date = @report_end_date 
        ) user
    ON a.aid = user.adobe_tracking_id
)
, EOM_Paid_Churn_Denom AS (
    SELECT  distinct adobe_tracking_id
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_CHURN`
    WHERE base_date = @report_end_date -- churn date would be on 11-30
    AND entitlement = 'Paid'
)
, EOM_Paid_Churn_Num AS (
    SELECT  distinct adobe_tracking_id
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_CHURN`
    WHERE base_date = @report_end_date
    AND entitlement = 'Paid'
    AND Churn_flag = 'Churn'
)
, cte1 as (
    SELECT cohort
        ,aid
        ,EOM_Paid_Churn_Denom.adobe_tracking_id  AS Churn_Denom
        ,EOM_Paid_Churn_Num.adobe_tracking_id    AS Churn_Num
    FROM base a
    LEFT JOIN EOM_Paid_Churn_Denom
        on a.aid = EOM_Paid_Churn_Denom.adobe_tracking_id
    LEFT JOIN EOM_Paid_Churn_Num
        on EOM_Paid_Churn_Denom.adobe_tracking_id = EOM_Paid_Churn_Num.adobe_tracking_id
    WHERE paying_account_flag = 'Paying'
    --and billing_cycle = 'MONTHLY'
)
, cte2 as (
    SELECT cohort
        ,COUNT(distinct Churn_Denom )   AS EOM_Paid_Churn_Denom_Targeted
        ,COUNT(distinct Churn_Num )     AS EOM_Paid_Churn_Num_Targeted
    FROM cte1
    group by 1
)
SELECT  cohort
       ,EOM_Paid_Churn_Denom_Targeted
       ,EOM_Paid_Churn_Num_Targeted
       ,safe_divide(EOM_Paid_Churn_Num_Targeted,EOM_Paid_Churn_Denom_Targeted) AS churn_rate
FROM cte2

Query is running:   0%|          |

Downloading:   0%|          |

,cohort,EOM_Paid_Churn_Denom_Targeted,EOM_Paid_Churn_Num_Targeted,churn_rate
0,Email_Targeted,10231349,595247,0.058179
1,Holdout,435022,26846,0.061712


✅ Channel version results

# Wes version

## Holdout

In [207]:
params = {
    'report_start_date': '2022-10-01',
    'report_end_date': '2022-10-01'
}

In [208]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Q42022_Holdout_Wes` AS

WITH 
email_holdout as (
    SELECT  distinct holdout.adobe_tracking_id
    FROM
    (
        SELECT  distinct TrackingId AS adobe_tracking_id
        FROM `nbcu-ds-prod-001.PeacockDataMartMarketingGold.HOLDOUT_GROUP`
        WHERE cohort = format_timestamp('%B%Y', DATETIME_TRUNC(@report_start_date, QUARTER)) -- get cohort name as month of quarter start + year
        AND Hold_Out_Type_Current = 'Owned Email Holdout'
        AND DATE(TIMESTAMP(RegistrationDate), 'America/New_York') <= @report_end_date
    ) holdout
    -- Exclude those who are assigned to Email Holdout but actually received emails 1 month prior to holdout start (to include cooloff period)
    LEFT JOIN (
        SELECT DISTINCT adobe_tracking_id 
        FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
        WHERE event_name = 'Email Deliveries' 
        AND event_date = @report_end_date
        AND LOWER(campaign_name) NOT LIKE '%transactional%' -- Exclude transactional emails
    ) received
    ON holdout.adobe_tracking_id = received.adobe_tracking_id
    WHERE received.adobe_tracking_id IS NULL
)
, email_channel_qualifier as (
    -- Engagement: Deliveries 4 months before start of the holdout period, defined as start of quarter
    SELECT DISTINCT adobe_tracking_id AS aid 
    FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
    WHERE event_name = 'Email Deliveries'
    AND event_date BETWEEN DATE_SUB(DATETIME_TRUNC(@report_start_date, QUARTER), INTERVAL 4 MONTH) AND @report_end_date
    AND lower(campaign_name) NOT LIKE 'transactional%' -- Exclude transactional emails

    UNION ALL

    -- New users joining after 4 months before start of the cohort period
    SELECT DISTINCT adobe_tracking_id AS aid
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
    WHERE registration_date between DATE_SUB(@report_start_date, INTERVAL 4 MONTH) and @report_start_date
)
, email_unsubs as (
    SELECT DISTINCT adobe_tracking_id
    FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE` 
    WHERE event_name = 'Email Unsubscribes' 
    AND event_date <= @report_end_date
)

SELECT  distinct email_holdout.adobe_tracking_id AS aid
FROM email_holdout

-- Include only those who received email in the current reporting period or are in holdout
INNER JOIN email_channel_qualifier
ON email_holdout.adobe_tracking_id = email_channel_qualifier.aid

-- for after 2021/july, email channel only, take out all abandon MAAs
INNER JOIN
    (
        SELECT  DISTINCT adobe_tracking_id
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES`
        WHERE report_date = @report_end_date
        AND date_of_last_view IS NOT NULL
    ) abandon_maa
ON email_holdout.adobe_tracking_id = abandon_maa.adobe_tracking_id

-- exclude unsubscribed
LEFT JOIN email_unsubs
ON email_holdout.adobe_tracking_id = email_unsubs.adobe_tracking_id
WHERE email_unsubs.adobe_tracking_id IS NULL

Query is running:   0%|          |

""


In [209]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

select count(distinct aid) from `nbcu-ds-sandbox-a-001.SLi_sandbox.Q42022_Holdout_Wes`

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,1962559


In [210]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

select count(distinct aid) from `nbcu-ds-sandbox-a-001.wes_crm_sandbox.Q42022_Holdout`

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,1962559


Holdout group size - Wes

In [211]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.churn_oct22_wes` AS

WITH EmailDeliveryStatus AS (
select 
  distinct '2022-10-01' Month_Year, 
  adobe_tracking_id aid, 
  event_name 
from `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
where 
  date(eventTimestamp) between '2022-10-01' and '2022-10-31'
  and event_name in('Email Deliveries') 
  and lower(campaign_name) not like '%transactional%' -- remove transactional emails

union all

select 
  distinct '2022-10-01' Month_Year, 
  adobe_tracking_id aid, 
  event_name -- identify if the last action a sub took was to unsubscribe
from `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
where 
  date(eventTimestamp) < '2022-10-31'
  and event_name in('Email Unsubscribes')
)
select 
  '2022-10-01' Month_Year,
  l.aid,
  l.billing_platform,
  l.billing_cycle,
  tenure_paid_lens,
  case when abandoned_ids is not null then "x" else null end abandoned_flag,
  marketing_status.category,
  case when l.billing_platform = 'NBCU' then 'Direct' else 'IAP' end grouped_billing_platform,
  case when l.paying_account_flag != r.paying_account_flag then 'Downgrade' else 'No Change' end change_flag,
  case when l.voucher_partner is null then "Not On Voucher" else "On Voucher" end voucher_flag,
  case when ho.aid is not null then "Holdout" when ho1.hid is not null then "Exclude" when l.aid is not null then 'Targetable' end Audience
from(
  select 
    adobe_tracking_id aid,
    household_id hid,
    billing_platform,
    paying_account_flag,
    billing_cycle,
    tenure_paid_lens,
    voucher_partner
  from `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
  where 
    report_date = '2022-10-31'
    and paying_account_flag = 'Paying'
) l

-- retrieve abandoned maas
left join(
  select adobe_tracking_id abandoned_ids
  from `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES` 
  where 
    report_date = '2022-10-31'
    and date_of_last_view is null
) a
on l.aid = a.abandoned_ids

-- retrieve thier marketing status: where they are unsubscribed or received an email in the last 4 months
left join(
  select 
    aid, 
    event_name category -- identify if the last action a sub took was to unsubscribe
  from EmailDeliveryStatus
  where Month_Year = '2022-10-01'
) marketing_status
on l.aid = marketing_status.aid

-- retrieve status 1 month later
left join(
  select 
    adobe_tracking_id,
    billing_platform,
    paying_account_flag
  from `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
  where 
    report_date = '2022-11-30'
) r
on l.aid = r.adobe_tracking_id

-- retrieve members of holdout group
left join(
  select aid
  from `nbcu-ds-sandbox-a-001.SLi_sandbox.Q42022_Holdout_Wes`
) ho
on l.aid = ho.aid

--retrieve the rest of the global hold out that doesnt fit holdout definition. exclude these users from the analysis
left join(
  select HouseholdId hid
  from `nbcu-ds-prod-001.PeacockDataMartMarketingGold.HOLDOUT_GROUP` 
  where 
    cohort = 'October2022' 
    and Hold_Out_Type_Current = 'Owned Email Holdout' -- Exclude those who are assigned to Email Holdout but actually received emails
    and TrackingId not in(
      select aid
      from `nbcu-ds-sandbox-a-001.SLi_sandbox.Q42022_Holdout_Wes`
    )
) ho1
on l.hid = ho1.hid

Query is running:   0%|          |

""


In [245]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

SELECT  Month_Year
       ,audience
       ,COUNT(distinct aid) paid_maas
       ,COUNT(distinct CASE WHEN change_flag = 'Downgrade' THEN aid else null end) churns
       ,COUNT(distinct CASE WHEN change_flag = 'Downgrade' THEN aid else null end) / COUNT(distinct aid) AS churn_rate
FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.churn_oct22_wes`
WHERE (((category != 'Email Unsubscribes') AND (audience = 'Targetable')) or (audience = 'Holdout'))
AND (abandoned_flag is null )
--AND billing_cycle = 'MONTHLY'
GROUP BY  1,2

Query is running:   0%|          |

Downloading:   0%|          |

,Month_Year,audience,paid_maas,churns,churn_rate
0,2022-10-01,Targetable,10283871,600898,0.058431
1,2022-10-01,Holdout,408121,23047,0.056471


✅ results - wes's logic

## Edited Version

In [14]:
params = {
    'report_start_date': '2022-10-01',
    'report_end_date': '2022-10-31'
}

In [15]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

create or replace table `nbcu-ds-sandbox-a-001.SLi_sandbox.Wes_Holdout_Test` AS

WITH 
email_holdout as (
    SELECT  distinct holdout.adobe_tracking_id
    FROM
    (
        SELECT  distinct TrackingId AS adobe_tracking_id
        FROM `nbcu-ds-prod-001.PeacockDataMartMarketingGold.HOLDOUT_GROUP`
        WHERE cohort = format_timestamp('%B%Y', DATETIME_TRUNC(@report_start_date, QUARTER)) -- get cohort name as month of quarter start + year
        AND Hold_Out_Type_Current = 'Owned Email Holdout'
        AND DATE(TIMESTAMP(RegistrationDate), 'America/New_York') <= @report_end_date
    ) holdout
    LEFT JOIN (
        SELECT DISTINCT adobe_tracking_id 
        FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
        WHERE event_name = 'Email Deliveries' 
        AND event_date BETWEEN DATETIME_TRUNC(@report_start_date, QUARTER) and @report_end_date -- CHANGED
        AND LOWER(campaign_name) NOT LIKE '%transactional%' -- Exclude transactional emails
    ) received
    ON holdout.adobe_tracking_id = received.adobe_tracking_id
    WHERE received.adobe_tracking_id IS NULL
)
, email_channel_qualifier as (
    -- Engagement: Deliveries 4 months before start of the holdout period, defined as start of quarter
    SELECT DISTINCT adobe_tracking_id AS aid 
    FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
    WHERE event_name = 'Email Deliveries'
    AND event_date BETWEEN DATE_SUB(DATETIME_TRUNC(@report_start_date, QUARTER), INTERVAL 4 MONTH) AND @report_end_date
    AND lower(campaign_name) NOT LIKE 'transactional%' -- Exclude transactional emails

    UNION ALL

    -- New users joining after 4 months before start of the cohort period
    SELECT DISTINCT adobe_tracking_id AS aid
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
    WHERE registration_date between DATE_SUB(@report_start_date, INTERVAL 4 MONTH) and @report_end_date -- CHANGED
)
, email_unsubs as (
    SELECT DISTINCT adobe_tracking_id
    FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE` 
    WHERE event_name = 'Email Unsubscribes' 
    AND event_date <= @report_end_date
)

SELECT  distinct email_holdout.adobe_tracking_id AS aid
FROM email_holdout

-- Include only those who received email in the current reporting period or are in holdout
INNER JOIN email_channel_qualifier
ON email_holdout.adobe_tracking_id = email_channel_qualifier.aid

-- for after 2021/july, email channel only, take out all abandon MAAs
INNER JOIN
    (
        SELECT  DISTINCT adobe_tracking_id
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES`
        WHERE report_date = @report_end_date
        AND date_of_last_view IS NOT NULL
    ) abandon_maa
ON email_holdout.adobe_tracking_id = abandon_maa.adobe_tracking_id

-- exclude unsubscribed
LEFT JOIN email_unsubs
ON email_holdout.adobe_tracking_id = email_unsubs.adobe_tracking_id
WHERE email_unsubs.adobe_tracking_id IS NULL

Query is running:   0%|          |

""


In [16]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

select count(distinct aid) from `nbcu-ds-sandbox-a-001.SLi_sandbox.Wes_Holdout_Test`

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,2018322


Holdout group size - Sanders

In [17]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.EmailDeliveryStatus_edit` AS

    SELECT  distinct '2022-10-01' Month_Year
        ,adobe_tracking_id aid
        ,event_name
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
    WHERE date(eventTimestamp) BETWEEN '2022-10-01' AND '2022-10-31'
    AND event_name in('Email Deliveries')
    AND lower(campaign_name) not like '%transactional%' -- remove transactional emails 
    UNION ALL
    SELECT  distinct '2022-10-01' Month_Year
        ,adobe_tracking_id aid
        ,event_name -- identify if the last action a sub took was to unsubscribe
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
    WHERE date(eventTimestamp) < '2022-10-31'
    AND event_name in('Email Unsubscribes') 

Query is running:   0%|          |

""


In [18]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

select count(distinct aid) from `nbcu-ds-sandbox-a-001.SLi_sandbox.EmailDeliveryStatus_edit` 

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,57705442


Email Delivery Status table

In [19]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.churn_oct22_wes_edit_v2` AS

WITH EmailDeliveryStatus AS 
(
    SELECT  distinct '2022-10-01' Month_Year
        ,adobe_tracking_id aid
        ,event_name
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
    WHERE date(eventTimestamp) BETWEEN '2022-10-01' AND '2022-10-31'
    AND event_name in('Email Deliveries')
    AND lower(campaign_name) not like '%transactional%' -- remove transactional emails 
    UNION ALL
    SELECT  distinct '2022-10-01' Month_Year
        ,adobe_tracking_id aid
        ,event_name -- identify if the last action a sub took was to unsubscribe
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
    WHERE date(eventTimestamp) <= '2022-10-31'
    AND event_name in('Email Unsubscribes') 
)
SELECT  '2022-10-01' Month_Year
       ,l.aid
       ,l.billing_platform
       ,l.billing_cycle
       ,tenure_paid_lens
       ,CASE WHEN abandoned_ids is not null THEN "x"  ELSE null END abandoned_flag
       ,marketing_status.category
       ,CASE WHEN l.billing_platform = 'NBCU' THEN 'Direct'  ELSE 'IAP' END grouped_billing_platform
       ,CASE WHEN l.paying_account_flag != r.paying_account_flag THEN 'Downgrade'  ELSE 'No Change' END change_flag
       ,CASE WHEN l.voucher_partner is null THEN "Not On Voucher"  ELSE "On Voucher" END voucher_flag
       ,CASE WHEN ho.aid is not null THEN "Holdout"
             WHEN ho1.hid is not null THEN "Exclude"
             WHEN l.aid is not null THEN 'Targetable' END Audience
FROM
(
	SELECT  adobe_tracking_id aid
	       ,household_id hid
	       ,billing_platform
	       ,paying_account_flag
	       ,billing_cycle
	       ,tenure_paid_lens
	       ,voucher_partner
	FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
	WHERE report_date = '2022-10-31'
	AND paying_account_flag = 'Paying' 
) l

-- retrieve abandoned maas
LEFT JOIN
(
	SELECT  adobe_tracking_id abandoned_ids
	FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES`
	WHERE report_date = '2022-10-31'
	AND date_of_last_view is null 
) a
ON l.aid = a.abandoned_ids

-- retrieve thier marketing status: where they are unsubscribed or received an email in the last 4 months
LEFT JOIN
(
	SELECT  aid
	       ,event_name category -- identify if the last action a sub took was to unsubscribe
	FROM EmailDeliveryStatus
	WHERE Month_Year = '2022-10-01' 
) marketing_status
ON l.aid = marketing_status.aid

-- retrieve status 1 month later
LEFT JOIN
(
	SELECT  adobe_tracking_id
	       ,billing_platform
	       ,paying_account_flag
	FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
	WHERE report_date = '2022-11-30' 
) r
ON l.aid = r.adobe_tracking_id

-- retrieve members of holdout group
LEFT JOIN (
	SELECT  aid
	FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.Wes_Holdout_Test`
) ho
ON l.aid = ho.aid

--retrieve the rest of the global hold out that doesnt fit holdout definition. exclude these users from the analysis
LEFT JOIN
(
	SELECT  HouseholdId hid
	FROM `nbcu-ds-prod-001.PeacockDataMartMarketingGold.HOLDOUT_GROUP`
	WHERE cohort = 'October2022'
	AND Hold_Out_Type_Current = 'Owned Email Holdout' -- Exclude those who are assigned to Email Holdout but actually received emails
	AND TrackingId not in(
	SELECT  aid
	FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.Wes_Holdout_Test` )        
) ho1
ON l.hid = ho1.hid

Query is running:   0%|          |

""


In [20]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

SELECT  Month_Year
       ,audience
       ,COUNT(distinct aid) paid_maas
       ,COUNT(distinct CASE WHEN change_flag = 'Downgrade' THEN aid else null end) churns
       ,COUNT(distinct CASE WHEN change_flag = 'Downgrade' THEN aid else null end) / COUNT(distinct aid) AS churn_rate
FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.churn_oct22_wes_edit_v2`
WHERE (((category != 'Email Unsubscribes') AND (audience = 'Targetable')) or (audience = 'Holdout'))
AND (abandoned_flag is null )
--AND billing_cycle = 'MONTHLY'
GROUP BY  1,2

Query is running:   0%|          |

Downloading:   0%|          |

,Month_Year,audience,paid_maas,churns,churn_rate
0,2022-10-01,Targetable,10283871,600898,0.058431
1,2022-10-01,Holdout,435022,26846,0.061712


✅ edited results